In [1]:
%config Completer.use_jedi=False
%load_ext memory_profiler

In [2]:
import numpy as np
from numpy.fft import fft
from numpy import sin
import timeit
from functools import partial
import memory_profiler

# Namespace stuff

In [32]:
x = np.ones(5); x

array([1., 1., 1., 1., 1.])

In [39]:
def local_fn(x):
    for i in range(1000): x = sin(x)
        
def global_fn(x):
    for i in range(1000): x = np.sin(x)

In [45]:
timeit.timeit(partial(local_fn, x), number=1000)

0.7584169530000509

In [46]:
timeit.timeit(partial(global_fn, x), number=1000)

0.8008171579999726

In [40]:
%timeit local_fn(x)

702 µs ± 8.53 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [41]:
%timeit global_fn(x)

786 µs ± 15.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


# Generators

In [57]:
a = [1,2,3,4,5]
type(a), type(iter(a))

(list, list_iterator)

In [58]:
def myrange(x=10):
    i = 0
    while i < x:
        yield i
        i += 1
        
type(myrange()), type(iter(myrange()))

(generator, generator)

In [60]:
type([1 for i in range(10)]), type((1 for i in range(10)))

(list, generator)

In [65]:
%memit sum([1 for i in range(1000000)])

peak memory: 81.67 MiB, increment: 16.12 MiB


In [66]:
%memit sum((1 for i in range(1000000)))

peak memory: 81.64 MiB, increment: -0.04 MiB


### Saving a lot of memory

In [83]:
from copy import copy
from itertools import chain
from statistics import mean

In [107]:
%time sum(x**2 for x in range(int(1e6)))

CPU times: user 389 ms, sys: 5.15 ms, total: 394 ms
Wall time: 396 ms


333332833333500000

In [108]:
%memit sum(x**2 for x in range(int(1e6)))

peak memory: 74.73 MiB, increment: 0.00 MiB


In [109]:
%time sum([x**2 for x in list(range(int(1e6)))])

CPU times: user 395 ms, sys: 43.2 ms, total: 438 ms
Wall time: 440 ms


333332833333500000

In [110]:
%memit sum([x**2 for x in list(range(int(1e6)))])

peak memory: 145.96 MiB, increment: 71.23 MiB


In [111]:
%%memit
nums = range(int(1e6))
odd_nums = filter(lambda x: x % 2 == 1, copy(nums))
eve_nums = filter(lambda x: x % 2 == 0, copy(nums))
sq_odd = (x**2 for x in odd_nums)
sq_eve = (x**2 for x in eve_nums)
finals = chain(sq_odd, sq_eve)
m      = sum(finals)

peak memory: 74.98 MiB, increment: 0.00 MiB


In [112]:
%%memit
nums = list(range(int(1e6)))
odd_nums = [x for x in nums if x % 2 == 1]
eve_nums = [x for x in nums if x % 2 == 0]
sq_odd = [x**2 for x in odd_nums]
sq_eve = [x**2 for x in eve_nums]
finals = sq_odd + sq_eve
m      = mean(finals)

peak memory: 171.02 MiB, increment: 96.29 MiB


### Infinite list

In [73]:
import gc
gc.collect()

35

In [74]:
# Infinite list with no memory impact at all
def inf_list(start=0):
    while True:
        yield start
        start += 1

### Itertools islice

In [76]:
from itertools import islice

In [77]:
a = range(100)
b = islice(a, 10)

In [78]:
list(b)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]